In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import math
import numpy as np
import matplotlib.colors as colors
import seaborn as sns
import itertools
from numpy import mean
from numpy import std

In [2]:
df = pd.read_csv('iris.csv') 
X = df.iloc[:, :-1]
y = df.iloc[:, 4].replace({'setosa':0,'versicolor':1,'virginica':1}).copy()
df['species']=y
print("Input Variables: \n", X)
print("Output Variables: \n", y)

Input Variables: 
      sepal_length  sepal_width  petal_length  petal_width
0             5.1          3.5           1.4          0.2
1             4.9          3.0           1.4          0.2
2             4.7          3.2           1.3          0.2
3             4.6          3.1           1.5          0.2
4             5.0          3.6           1.4          0.2
..            ...          ...           ...          ...
145           6.7          3.0           5.2          2.3
146           6.3          2.5           5.0          1.9
147           6.5          3.0           5.2          2.0
148           6.2          3.4           5.4          2.3
149           5.9          3.0           5.1          1.8

[150 rows x 4 columns]
Output Variables: 
 0      0
1      0
2      0
3      0
4      0
      ..
145    1
146    1
147    1
148    1
149    1
Name: species, Length: 150, dtype: int64


In [3]:
class NaiveBayes():
    
    def __init__(self):
        self.df = pd.read_csv('iris.csv', sep=',', header=None)
        self.X = df.iloc[:, :-1]
        self.y = df.iloc[:, 4].replace({'setosa':0,'versicolor':1,'virginica':1}).copy()
        self.df['species'] = self.y
        self.prior_probability = []
        self.pred = []
        self.prob = []
        self.post_prod = []
        self.mean1 = []
        self.std1 = []
    
    def bins(self, X, i):
        attribute = np.array(X.columns)

        for k in attribute:

            count, bins = np.histogram(X[k], bins=i)
            
            mids = 0.5*(bins[1:] + bins[:-1])
            
            mean = np.average(mids, weights=count)
            self.mean1 += (mean, round(mean,3))
            
            var = np.average((mids - mean)**2, weights=count)
            
            std = np.sqrt(var)
            self.std1 += (std, round(var,3))

        return self.mean1, self.std1
    
    def split_data(self, X, y):
        X_train, X_test, Y_train, Y_test = train_test_split(self.X, self.y, test_size = 0.33)
        return X_train, X_test, Y_train, Y_test
        
    def naive(self, X_train, X_test, Y_train, Y_test, i):
        data0 = X_train[Y_train == 0] 
        data1 = X_train[Y_train == 1] 
        print(data0.shape, data1.shape)
        
        prior0 = len(data0) / len(X_train)
        prior1 = len(data1) / len(X_train)
        self.prior_probabilty = [prior0, prior1]
        
        ms0 = self.bins(data0, i)
        ms1 = self.bins(data1, i)
        
        info = {0:ms0,1:ms1}
              
        prediction, probability = self.predictions(info, X_test, self.prior_probability)
        
        print(prior0, prior1)
        print(info)
              
        return prediction, probability
    
    def predictions(self, info, X_test, prior_probabilty):
        for i in range(len(X_test)):
            k = X_test.iloc[[i]].values.flatten().tolist()
            
            b, c = self.pred1(info, k, prior_probabilty)
            
            self.pred += (b)
            self.prob += (c)
        
        return pred, prob
    
    def pred1(info, test, prior_probabilty):
        
        self.post_prob = self.cal_post_prob(info, test, prior_probabilty)
        
        bestLabel, bestProb = None, -1
        
        for classValue, probability in self.post_prob.items():
            if(bestLabel is None or probability > bestProb):
                bestProb = probability
                bestLabel = classValue
                
        return bestLabel, bestProb
    
    def cal_post_prob(info, test, prior_probabilty):
        probabilities = {}
        
        for i, j in info.items():
            probabilities[i] = 1
            
            for k in range(len(j[0])):

                m = j[0][k]

                s = j[1][k]

                x = test[k]
                
                g = self.gprob(x, m, s)

                probabilities[i] *= g
            
            probabilities[i] = probabilities[i] * self.prior_probabilty[i]

        return probabilities
    
    def gprob(self, x, m, s):
        mu = math.pow(x-m ,2)
        sig2 = (2 * math.pow(s, 2))
        expo = math.exp(-(mu/sig2))
        gsd = (1/(math.sqrt((2*math.pi) *s)) * expo)
        return gsd


In [55]:
class Metrics():
    def compute_tp_tn_fn_fp(y_test, y_pred):
        tp=0
        tn=0
        fn=0
        fp=0
        y_test=y_test.tolist()
        for i in range(len(y_act)):
            if y_test[i]==1 and y_pred[i]==1:
                tp+=1
            elif y_test[i]==0 and y_pred[i]==0:
                tn+=1
            elif y_test[i]==1 and y_pred[i]==0:
                fn+=1
            elif y_test[i]==0 and y_pred[i]==1:
                fp+=1

        return tp, tn, fp, fn

    def computemetrics(y_true, y_pred):
        tp, tn, fp, fn = compute_tp_tn_fn_fp(y_true, y_pred)

        precision = cal_precision(tp, fp)/100

        accuracy =cal_accuracy(tp,tn,fn,fp)

        recall = cal_recall(tp, fn)/100

        f1_score = ((2*precision*recall)/ (precision + recall))

        tpr=tp/(tp+fn)

        fpr=fp/(fp+tn)

        return f1_score, accuracy, tpr, fpr


    def plotAccuracies(bins,accuracies):
        plt.figure(figsize=(16,8))
        acc = [i+1 for i in range(len(accuracies[0]))]
        axes = plt.axes()
        axes.set_title('Naivebayes Accuracy Plot')
        axes.set_xlabel('Testing Set')
        axes.set_xticks(acc)
        axes.set_ylabel('Accuracy (%)')
        axes.plot(acc, accuracies[0],label = 'Bins = 5', marker='o', markersize=25)
        axes.plot(acc, accuracies[1],label = 'Bins = 10', marker='o', markersize=20)
        axes.plot(acc, accuracies[2],label = 'Bins = 15', marker='o', markersize=15)
        axes.plot(acc, accuracies[3],label = 'Bins = 20', marker='o', markersize=10)   
        axes.legend()
        plt.show()


    def plotF1(bins,f1_scores):
        plt.figure(figsize=(16,8))
        f1 = [i+1 for i in range(len(f1_scores[0]))]
        axes = plt.axes()
        axes.set_title('Naivebayes'+' - F1 Score Plot')
        axes.set_xlabel('Testing Set')
        axes.set_xticks(f1)
        axes.set_ylabel('F1 Score')
        axes.plot(f1, f1_scores[0],label = 'Bins = 5',marker='o',markersize=25)
        axes.plot(f1, f1_scores[1],label = 'Bins = 10',marker='o',markersize=20)
        axes.plot(f1, f1_scores[2],label = 'Bins = 15',marker='o',markersize=15)
        axes.plot(f1, f1_scores[3],label = 'Bins = 20',marker='o',markersize=10)   
        axes.legend()
        plt.show()

    def cal_recall(tp, fn):
        return (tp * 100)/ float( tp + fn)

    def cal_precision(tp, fp):
        return (tp * 100)/ float( tp + fp)

    def cal_accuracy(tp,tn,fn,fp):
        return((tp+tn)/(tp+tn+fp+fn))*100

NameError: name 'plotAccuracies' is not defined

In [59]:
object = NaiveBayes()
object1 = Metrics()
bins = [5, 10, 15, 20]

X_train1, X_test1, y_train1, y_test1 = object.split_data(object.X, object.y)
X_train2, X_test2, y_train2, y_test2 = object.split_data(object.X, object.y)
X_train3, X_test3, y_train3, y_test3 = object.split_data(object.X, object.y)
X_train4, X_test4, y_train4, y_test4 = object.split_data(object.X, object.y)

for i in bins:
    mu, stdi = object.bins(object.X, i)
    npred1, nprob1 = object.naive(X_train1, X_test1, y_train1, y_test1, i)
    stats1 = object1.compstats(y_test1, npred1)
    accuracy1 = stats1[1]


(32, 4) (68, 4)


TypeError: pred1() takes 3 positional arguments but 4 were given